In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from api_keys import api_key
from datetime import datetime
from time import mktime

#need to fix
key ="54445e5623750bd7435f05cf140903c3"

In [2]:
#read data 
LR_csv = pd.read_csv("cleaned_data.csv")
LR_csv.head()

,MONTH,SUPPLIER,FACTORY,COUNTRY,DPN,LOB,WATT_LEVEL,LR,CONSUMPTION,VID,CND,PID
0,JA,A,Brasilia,Brazil,7FHHV,Consumer,Medium,0,35,0,0,0
1,JA,A,Brasilia,Brazil,7FHHV,Consumer,Medium,0,50,0,0,0
2,JA,A,Brasilia,Brazil,7FHHV,Commercial,Medium,0,158,0,0,0
3,JA,A,Chengdu,China,7FHHV,Consumer,Medium,0,1,0,0,0
4,JA,A,Chengdu,China,7FHHV,Commercial,Medium,0,1,0,0,0


In [3]:
# making list for months and cities
cities = []
months = []
mons = []
for city in LR_csv["FACTORY"]:
    cities.append(city)
#gathering months and converting to numbers in the list    
for month in LR_csv["MONTH"]:
    mons.append(month)
    if month == "JA":
        months.append(1)
    if month == "FE":
        months.append(2)
    if month == "MR":
        months.append(3)
    if month == "AP":
        months.append(4)
    if month == "MY":
        months.append(5)

cities = list(dict.fromkeys(cities))
months = list(dict.fromkeys(months))
mons = list(dict.fromkeys(mons))
cities

['Brasilia', 'Chengdu', 'Kunshan', 'Penang', 'Mumbai', 'Suzhou', 'Taoyuan']

In [4]:
# getting long and lat
lat = []
lng = []
for city in cities:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=imperial&APPID={api_key}"
    url = requests.get(url).json()   
    lat.append(url["coord"]["lat"])
    lng.append(url["coord"]["lon"])


In [5]:
#creating new data frame with cities with lat and long (for easier discorvery)
city_coord_df = pd.DataFrame({"City":cities,"Latitude":lat,"Longitude":lng})
city_coord_df

,City,Latitude,Longitude
0,Brasilia,-10.90,-37.55
1,Chengdu,30.68,104.06
2,Kunshan,31.39,120.95
3,Penang,-7.81,113.99
4,Mumbai,19.01,72.85
5,Suzhou,33.65,116.96
6,Taoyuan,24.99,121.30


In [6]:
#converting to unix timestamp

t = datetime(2018, 1, 1)
unix_secs = mktime(t.timetuple())
dt = str(int(unix_secs))

#checking it worked
dt = 1559278800
date = datetime.utcfromtimestamp(int(dt)).strftime('%Y-%m-%d')

print(dt, date)

1559278800 2019-05-31


In [7]:
dts = []
i = 1
j = 1
for i in months:
    dts.append((2019, i, 1))
    if i == 1:
        j += 30
        dts.append((2019, i, j))
        j = 1
    if i == 2:
        j += 27
        dts.append((2019, i , j))
        j = 1
    if i == 3:
        j += 30
        dts.append((2019, i, j))
        j = 1
    if i == 4:
        j += 29
        dts.append((2019, i, j))
        j = 1
    if i == 5:
        j += 30
        dts.append((2019, i, j))
len(dts)

10

In [8]:
#making list of the start and then end of each months dt
unix_dts = []
x = 0
for x in range(len(dts)): 
    t = datetime(dts[x][0],dts[x][1],dts[x][2])
    unix_dt = mktime(t.timetuple())
    unix_dt = str(int(unix_dt))
    unix_dts.append(unix_dt)
unix_dts[0]

'1546322400'

In [9]:
#variables for the upcoming dataframe
temps = ['a','b','c','d','e','f','g']
i = 0
j = 0
col_names = ["JAstrt","JAend","FEstrt","FEend","MRstrt","MRend","APstrt","APend","MYstrt","MYend"]
temp_df = pd.DataFrame()
#nameing columns for temps
for mon in mons:
    count = 0
    for i in range(len(unix_dts)):

        if count%2 == 0:
            col = mon + "strt"
            city_coord_df[col] = temps
            count +=1
        else:
            col = mon + "end"
            city_coord_df[col] = temps
            count += 1
            
#pulling temp from unix dts            
for i in range(0,len(unix_dts)):
    d = unix_dts[i]
    temps = []

    
    for j in range(0,len(city_coord_df["City"])):
        lt = city_coord_df["Latitude"][j]
        lg = city_coord_df["Longitude"][j]
        drk_sky = f"https://api.darksky.net/forecast/54445e5623750bd7435f05cf140903c3/{lt},{lg},{d}?exclude=flags,hourly,minutely,alerts,daily"
        drk_sky = requests.get(drk_sky).json()
        temps.append(drk_sky["currently"]["temperature"])
    col = col_names[i]
    city_coord_df[col] = temps
        

city_coord_df

,City,Latitude,Longitude,JAstrt,JAend,FEstrt,FEend,MRstrt,MRend,APstrt,APend,MYstrt,MYend
0,Brasilia,-10.90,-37.55,73.49,72.62,73.16,72.82,74.14,73.54,72.61,76.10,75.52,71.53
1,Chengdu,30.68,104.06,43.09,45.85,52.51,52.16,46.64,67.04,58.79,72.24,75.86,82.93
2,Kunshan,31.39,120.95,39.90,41.17,39.61,50.59,51.53,57.32,65.23,60.95,74.99,77.23
3,Penang,-7.81,113.99,85.35,86.31,84.46,87.86,85.45,88.85,85.73,84.12,85.04,91.94
4,Mumbai,19.01,72.85,84.70,82.02,81.89,82.51,83.99,85.01,84.63,89.15,88.88,89.21
5,Suzhou,33.65,116.96,39.34,35.29,44.75,52.87,53.41,61.95,70.77,73.83,78.10,86.17
6,Taoyuan,24.99,121.30,61.64,77.08,56.16,62.79,70.17,66.30,60.38,86.01,65.69,88.34


In [10]:
city_coord_df = city_coord_df[["City","Latitude","Longitude","JAstrt","JAend","FEstrt","FEend","MRstrt","MRend","APstrt","APend","MYstrt","MYend"]]
city_coord_df.to_csv("city_weather.csv")

In [11]:
city_coord_df

,City,Latitude,Longitude,JAstrt,JAend,FEstrt,FEend,MRstrt,MRend,APstrt,APend,MYstrt,MYend
0,Brasilia,-10.90,-37.55,73.49,72.62,73.16,72.82,74.14,73.54,72.61,76.10,75.52,71.53
1,Chengdu,30.68,104.06,43.09,45.85,52.51,52.16,46.64,67.04,58.79,72.24,75.86,82.93
2,Kunshan,31.39,120.95,39.90,41.17,39.61,50.59,51.53,57.32,65.23,60.95,74.99,77.23
3,Penang,-7.81,113.99,85.35,86.31,84.46,87.86,85.45,88.85,85.73,84.12,85.04,91.94
4,Mumbai,19.01,72.85,84.70,82.02,81.89,82.51,83.99,85.01,84.63,89.15,88.88,89.21
5,Suzhou,33.65,116.96,39.34,35.29,44.75,52.87,53.41,61.95,70.77,73.83,78.10,86.17
6,Taoyuan,24.99,121.30,61.64,77.08,56.16,62.79,70.17,66.30,60.38,86.01,65.69,88.34
